In [ ]:
start = time();

In [ ]:
using CUDA
using CSV
using DataFrames
using Printf
using Statistics
using Juliana

In [ ]:
CUDA.devices()

# Configuration

In [ ]:
data_dir = "/data/user/bellotti_r/data_special_cases/EPTN";
target_dose = 69.96f0; # Gy RBE

fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.10.jar";
output_dir = "../output/demo_notebook_EPTN"

mkpath(output_dir);

# Load data

In [ ]:
"""
    transform_angles_to_G2_range(gantry_angle, couch_angle)

Input ranges:
gantry_angle ∈ [0, 360]
couch_angle ∈ [0, 360]

Output ranges:
gantry_angle ∈ [-30, 180]
couch_angle ∈ [-180, 180]
"""
function transform_angles_to_G2_range(gantry_angle, couch_angle)
    @assert 0 <= gantry_angle && gantry_angle <= 360
    @assert 0 <= couch_angle && couch_angle <= 360
    if !(0 <= gantry_angle && gantry_angle <= 180)
        gantry_angle = 360 - gantry_angle
        couch_angle = couch_angle + 180
    end
    if couch_angle > 180
        couch_angle = couch_angle - 360
    end
    
    return gantry_angle, couch_angle
end


test_cases = [
    ((190, 0), (170, 180)),
    ((  0, 0), (  0,   0)),
    ((200, 0), (160, 180)),
    ((200, 90), (160, -90)),
    ((200, 70), (160, -110)),
    ((240, 70), (120, -110)),
    ((240, 0), (120, 180)),
]
for (input, output) in test_cases
    trafo_g, trafo_c = transform_angles_to_G2_range(input[1], input[2])
    @assert (trafo_g == output[1]) && (trafo_c == output[2])
end


function load_fields_description(beam_arr_filename)
    # Values from Eclipse.
    fields = DataFrame(CSV.File(beam_arr_filename))

    # Transform to G2 angles.
    for row in eachrow(fields)
        g, c = transform_angles_to_G2_range(
            row["gantry_angle"],
            row["couch_angle"],
        )
        row["gantry_angle"] = g
        row["couch_angle"] = c
    end

    # Angles for G2.
    return fields
end


"""
    field_df_to_juliana(fields, structures)

Convert a DataFrame of the field configuration to Juliana structures.
Return the field configuration as a vector of fields (without spots)
and an array that contains the structures to be used as the field centers.
"""
function field_df_to_juliana(fields, structures)
    field_defs = Vector{Juliana.FionaStandalone.FieldDefinition{Float32}}(undef, size(fields, 1))
    for (i, row) in enumerate(eachrow(fields))
        center = mean(structures[row.target_name].points, dims=1)[1:3]
        field_defs[i] = Juliana.FionaStandalone.FieldDefinition(
            convert(String, row.name),
            i-1,
            convert(Float32, row["gantry_angle"]),
            convert(Float32, row["couch_angle"]),
            convert(Float32, row["nozzle_extraction"]),
            Dict{String, Float32}(
                "x" => center[1],
                "y" => center[2],
                "z" => center[3],
            ),
            Vector{Juliana.FionaStandalone.Spot{Float32}}(undef, 0),
        )
    end

    field_targets = [structures[f.target_name] for f in eachrow(fields)]
    
    return field_defs, field_targets
end


function load_fields(beam_arr_filename, structures)
    return field_df_to_juliana(
        load_fields_description(beam_arr_filename),
        structures,
    )
end

In [ ]:
@time ct, structures = Juliana.load_dicom_directory(
    data_dir;
    structure_names=[
        "BrainStem",
        "Chiasm",
        "Cochlea_L",
        "Cochlea_R",
        "Eye_Post_L",
        "Eye_Post_R",
        "OpticNerve_L",
        "OpticNerve_R",
        "Parotid_L",
        "Parotid_R",
        "SpinalCord",
        "Brain",
        "PTV1", "PTV2",
        "TECHPTV_UP", "TECHPTV_HYBL", "TECHPTV_HYBR",
        "TECHPTV_DWNL", "TECHPTV_DWNR",
    ],
);

In [ ]:
constraints = Juliana.parse_oar_constraints_file(
    "$(data_dir)/constraints.csv",
    target_dose,
    structures,
)
prescriptions = Juliana.Prescriptions(
    [
        ("PTV1", 69.96f0),
        ("PTV2", 59.40f0),
    ],
    constraints,
);

In [ ]:
tps = Juliana.FionaStandalone.FionaStandaloneTps(
    fiona_standalone_bin_path,
    fiona_jar_path,
    output_dir,
);

# Optimise

In [ ]:
fields, center_structures = load_fields("$(data_dir)/beam_arrangement.csv", structures);

In [ ]:
@time plan = Juliana.FionaStandalone.place_spots(
    tps,
    ct,
    fields,
    center_structures,
);

In [ ]:
@time optimal_weights, gradients, history = Juliana.optimise_head_and_neck(ct, structures, prescriptions, plan, tps, tps.work_dir);
plan_optimised = Juliana.FionaStandalone.update_spot_weights(plan, optimal_weights);

# Calculate the dose distribution

In [ ]:
dose = Juliana.FionaStandalone.calculate_dose(
    tps,
    0.35f0, # dose grid resolution
    ct,
    plan_optimised,
);

In [ ]:
stop = time();

In [ ]:
@printf "Time to run the demo notebook: %.2fs" stop - start

# Reports

In [ ]:
report = Juliana.Report("report", dose, prescriptions, structures);

In [ ]:
Juliana.constraint_fulfillment_df([report])

In [ ]:
Juliana.target_coverage_df([report])

In [ ]:
length(plan_optimised.fields)

In [ ]:
sum([length(f.spots) for f in plan_optimised.fields])

# Export to DICOM

In [ ]:
patient_ID = "INSPIRE_2"

In [ ]:
study_instance_UID = Juliana.get_study_instance_uid(patient_ID);

In [ ]:
filenames = readdir(data_dir)

ct_files = [f for f in filenames if startswith(f, "CT.")]
ct_files = sort(ct_files, by=f -> parse(Int64, String(split(f, ".")[end-1])))
ct_paths = ["$(data_dir)/$(f)" for f in ct_files]
orig_ct = Juliana.read_dicom_ct(ct_paths)
            
# export_grid = ct.grid;
export_grid = orig_ct.grid;

In [ ]:
 Juliana.dicom_export_to_directory(
    ct,#     Juliana.ScalarGrid(ct.data, export_grid),
    structures,
    output_dir,
    study_instance_UID,
    patient_ID,
    Dict{String, Juliana.ScalarGrid}("juliana" => Juliana.ScalarGrid(dose.data, export_grid)),
)